In [1]:
import numpy as np



In [9]:
F = 0.003
G = 0.003

Q_ab = {'a': 100, 'b': 100}
Q_ac = {'a': 200, 'c': 200}
Q_bc = {'b': 250, 'c': 250}

In [ ]:
def psi(x):
    assert x >= 0
    return 1/(1+x)

def phi(x):
    assert x >= 0
    assert x < 1
    return 1/(1-x)

def forward_swap(d_in, in_leg , out_leg, q, fee):
    assert q[in_leg] > 0
    d_out = (1 - fee) * ( q[out_leg]/q[in_leg] ) * psi(d_in/q[in_leg]) * d_in
    q[out_leg] = q[out_leg] - d_out
    q[in_leg] = q[in_leg] + d_in
    return d_out, q

def backward_swap(d_out, in_leg, out_leg, q, fee):
    assert q[out_leg] > 0
    d_in = (1/(1-fee)) * ( q[in_leg]/q[out_leg] ) * phi(d_out/q[out_leg]) * d_out
    q[out_leg] = q[out_leg] - d_out
    q[in_leg] = q[in_leg] + d_in
    return d_in, q

In [23]:
F = 0.003
G = 0.003

Q_ab = {'a': 100, 'b': 100}
Q_ac = {'a': 200, 'c': 200}
Q_bc = {'b': 250, 'c': 250}
# a->b route
D_a1 = 10
D_b1, Q_ab = forward_swap(d_in=D_a1, in_leg='a', out_leg='b', q=Q_ab, fee=G)

print(D_a1, D_b1, Q_ab)

10 9.063636363636364 {'a': 110, 'b': 90.93636363636364}


In [24]:
F = 0.003
G = 0.003

Q_ab = {'a': 100, 'b': 100}
Q_ac = {'a': 200, 'c': 200}
Q_bc = {'b': 250, 'c': 250}
# a->c->b route
D_c2, Q_ac = forward_swap(d_in=D_a1, in_leg='a', out_leg='c', q=Q_ac, fee=G)
print(D_a1, D_c2, Q_ac)

D_b2, Q_bc = forward_swap(d_in=D_c2, in_leg='c', out_leg='b', q=Q_bc, fee=G)
print(D_c2, D_b2, Q_bc)



10 9.495238095238093 {'a': 210, 'c': 190.5047619047619}
9.495238095238093 9.120352699379746 {'b': 240.87964730062026, 'c': 259.4952380952381}


In [25]:
print(D_b1 - D_b2)

-0.05671633574338131


In [26]:
# reset reserves
Q_ab = {'a': 100, 'b': 100}
Q_ac = {'a': 200, 'c': 200}
Q_bc = {'b': 250, 'c': 250}

In [27]:
# cycle c -> b -> a -> c
print(D_c2)
C_b, Q_bc = forward_swap(D_c2, in_leg='c', out_leg='b', q=Q_bc, fee=G)
print(C_b, Q_bc)

9.495238095238093
9.120352699379746 {'b': 240.87964730062026, 'c': 259.4952380952381}


In [28]:
C_a, Q_ab = forward_swap(d_in=C_b, in_leg='b', out_leg='a', q=Q_ab, fee=G)
print(C_a, Q_ab)

8.332993265089852 {'a': 91.66700673491015, 'b': 109.12035269937975}


In [29]:
C_c, Q_ac = forward_swap(d_in=C_a, in_leg='a', out_leg='c', q=Q_ac, fee=G)
print(C_c, Q_ac)

7.975687532817438 {'a': 208.33299326508984, 'c': 192.02431246718257}
